### Download Dataset

In [ ]:
# !kaggle datasets download -d paramaggarwal/fashion-product-images-small
# !unzip fashion-product-images-small.zip

### Importação das bibliotecas

In [1]:
import os
import itertools
from pathlib import Path
from shutil import move
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm

# Extensão do tqdm para DataFrames
tqdm.pandas()

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

2025-02-20 16:31:52.217481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740079912.238551   11551 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740079912.244140   11551 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 16:31:52.262987: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF version: 2.18.0
Hub version: 0.16.1
GPU is available


### Preparação de diretórios e organização das imagens

In [2]:
# Criar pasta principal
data_dir = 'Fashion_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Ler o CSV
df = pd.read_csv('styles.csv', usecols=['id', 'masterCategory'])
df['id'] = df['id'].astype(str)

# Mover imagens para subpastas de categoria
images_source = 'images'
categories_dir = os.path.join(data_dir, 'categories')

# Criar diretório de categorias se não existir
os.makedirs(categories_dir, exist_ok=True)

# Processar cada imagem
all_images = os.listdir(images_source)
moved_count = 0

for image in tqdm(all_images):
    image_id = image.split('.')[0]
    category = df.loc[df['id'] == image_id, 'masterCategory'].values
    
    if len(category) == 0:
        continue  # Pula se a categoria não for encontrada
    
    category = category[0]
    category_path = os.path.join(categories_dir, category)
    
    # Criar subpasta da categoria se necessário
    os.makedirs(category_path, exist_ok=True)
    
    # Mover a imagem
    src = os.path.join(images_source, image)
    dst = os.path.join(category_path, image)
    move(src, dst)
    moved_count += 1

print(f'Total de imagens movidas: {moved_count}')

100%|██████████| 44441/44441 [02:30<00:00, 294.81it/s]

Total de imagens movidas: 44441


### Definição de parâmetros e modelo

In [3]:
MODULE_HANDLE = "https://tfhub.dev/google/bit/m-r50x3/1"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
N_FEATURES = 256

# Classe de encapsulamento do hub.KerasLayer
class HubWrapper(tf.keras.layers.Layer):
    def __init__(self, module_handle, trainable=False, **kwargs):
        super().__init__(**kwargs)
        # Define uma camada do TF Hub
        self.hub_layer = hub.KerasLayer(module_handle, trainable=trainable)

    def call(self, inputs):
        # Encaminha os dados de entrada para a camada do Hub
        return self.hub_layer(inputs)

# Construção do modelo Sequential
model = tf.keras.Sequential([
    # Camada de entrada
    tf.keras.layers.Input(shape=IMAGE_SIZE + (3,)),
    
    # Envolve a camada do TensorFlow Hub no wrapper personalizado
    HubWrapper(MODULE_HANDLE, trainable=False),
    
    # Camada de Dropout para reduzir overfitting
    tf.keras.layers.Dropout(rate=0.2),
    
    # Camada densa intermediária
    tf.keras.layers.Dense(
        N_FEATURES,
        kernel_regularizer=tf.keras.regularizers.l2(0.0001)
    ),
    tf.keras.layers.Dropout(rate=0.2),
    
    # Camada de saída com 7 neurônios, pois temos 7 classes
    tf.keras.layers.Dense(
        7,  # Ajustado para 7 classes
        kernel_regularizer=tf.keras.regularizers.l2(0.0001)
    )
])

# Constrói formalmente o modelo (opcional, mas útil para summary)
model.build((None,) + IMAGE_SIZE + (3,))
model.summary()

I0000 00:00:1740080484.803071   11551 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2795 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
2025-02-20 16:41:32.945689: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
2025-02-20 16:41:32.945729: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
2025-02-20 16:41:32.945747: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
2025-02-20 16:41:32.945763: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75497472 exceeds 10% of free system memory.
2025-02-20 16:41:33.213041: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 37748736 exceeds 10% of free system memory.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hub_wrapper (HubWrapper)        │ (None, 6144)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6144)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,573,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,574,919 (6.01 MB)

 Trainable params: 1,574,919 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

### Preparação dos geradores de dados(treino/validação)

In [ ]:
data_dir = 'Fashion_data/categories'

# Normalização e separação (20% para validação)
datagen_kwargs = dict(rescale=1./255, validation_split=0.20)

# Parâmetros para o fluxo de dados
dataflow_kwargs = dict(
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    interpolation="bilinear"
)

# Gerador de dados para o conjunto de validação
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir,
    subset="validation",
    shuffle=False,
    **dataflow_kwargs
)

# Gerador de dados para treinamento
# Define se vamos utilizar data augmentation
do_data_augmentation = False
if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagen_kwargs
    )
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    data_dir,
    subset="training",
    shuffle=True,
    **dataflow_kwargs
)

FileNotFoundError: [Errno 2] No such file or directory: '/Fashion_data/categories'